In [1]:
import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)

import tensorflow.keras as tfk
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Flatten, Reshape, Lambda
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Model
import tensorflow_probability as tfp
tfkl = tfk.layers
tfd = tfp.distributions
tfpl = tfp.layers
import os
import os.path as osp
import sys
sys.path.append('/home/jcollins')

import numpy as np
from scipy import linalg as LA

import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [47]:
from build_vae_oneparticle import make_vae_oneparticle, make_encoder_oneparticle

In [48]:
data_inputs = tf.keras.Input(shape=(2,))
beta_inputs = tf.keras.Input(shape=(1,))
inputs = tfkl.Concatenate()([data_inputs,beta_inputs])

In [61]:
dense_sizes = [64,64,64]
latent_dims_line = 0
latent_dims_circle = 2
layer = data_inputs

for layer_size in dense_sizes:
    layer = tfkl.Dense(layer_size,
                       activation='relu',
                       kernel_initializer='glorot_uniform',
                       bias_initializer='glorot_uniform')(layer)

gauss_z_mean = tfkl.Dense(latent_dims_line, name='gauss_z_mean',activation=None)(layer)
gauss_z_log_var = tfkl.Dense(latent_dims_line, name='gauss_z_log_var',activation=None)(layer)

vm_z_mean_x = tfkl.Dense(latent_dims_circle, name='vm_z_mean_x',activation=None)(layer)
vm_z_mean_y = tfkl.Dense(latent_dims_circle, name='vm_z_mean_y',activation=None)(layer)
vm_z_mean = tf.atan2(vm_z_mean_x,vm_z_mean_y)

In [62]:
vm_z_mean

<tf.Tensor 'Atan2_22:0' shape=(None, 2) dtype=float32>

In [53]:
encoder, distributions = make_encoder_oneparticle(data_inputs,
                                   latent_dims_line=2,
                                   latent_dims_circle=2,
                                   dense_sizes = [64,64,64])


In [54]:
distributions

[<tfp.distributions.MultivariateNormalDiag 'gauss_distribution_11_MultivariateNormalDiag' batch_shape=? event_shape=[2] dtype=float32>,
 <tfp.distributions.VonMises 'vm_distribution_11_VonMises' batch_shape=[?, 2] event_shape=[] dtype=float32>]

In [55]:
distributions[0].sample()

TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [int32, int32] that don't all match.

In [57]:
encoder.summary()

Model: "model_47"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           [(None, 2)]          0                                            
__________________________________________________________________________________________________
dense_102 (Dense)               (None, 64)           192         input_39[0][0]                   
__________________________________________________________________________________________________
dense_103 (Dense)               (None, 64)           4160        dense_102[0][0]                  
__________________________________________________________________________________________________
dense_104 (Dense)               (None, 64)           4160        dense_103[0][0]                  
___________________________________________________________________________________________

In [32]:
encoder.predict([np.array([[0.,0.],[10.,10.],[0.,-1.]]),np.array([1.,1.,1.])])

[array([[ 0.04333429,  0.00445133],
        [ 0.47419477, -0.5849418 ],
        [ 0.09433827, -0.05565662]], dtype=float32),
 array([[-0.22277287,  0.12485981],
        [-1.266582  ,  1.0127592 ],
        [-0.24177864,  0.14372826]], dtype=float32),
 array([[ 0.01614719,  1.8005726 ],
        [ 0.71239245, -1.3859606 ],
        [-0.21157822,  1.497665  ]], dtype=float32)]

In [33]:
encoder

In [36]:
tf.get_default_graph().get_tensor_by_name("gauss_distribution:0")

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [46]:
vae = make_vae_oneparticle(latent_dims_circle=1)

Model: "model_45"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, 2)]          0                                            
__________________________________________________________________________________________________
model_43 (Model)                [(None, 2), (None, 2 8837        input_37[0][0]                   
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
model_44 (Model)                (None, 2)            8642        model_43[1][2]                   
Total params: 17,479
Trainable params: 17,479
Non-trainable params: 0
_____________________